### 1. Pytorch 自动求导数

In [88]:
import torch
## 1. Inference
x= torch.ones(2,2,requires_grad = True)
print("*"*50,'\n x is : \n',x)

y = x + 2
print("*"*50,'\n y = x + 2 is :\n',y)

z = y * y  * 3
print("*"*50,'\n z = y * y *3 is :\n',z)

out = z.mean()
print("*"*50, '\n out = y.mean() is :\n', out)

## 2 . Backward - out is scalar
out.backward()
print("\n","*"*50, "\n x grad is:\n", x.grad)

************************************************** 
 x is : 
 tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
************************************************** 
 y = x + 2 is :
 tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
************************************************** 
 z = y * y *3 is :
 tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
************************************************** 
 out = y.mean() is :
 tensor(27., grad_fn=<MeanBackward0>)

 ************************************************** 
 x grad is:
 tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


In [90]:
## when output is vector 
x = torch.randn(3, requires_grad = True)
y = x * 2
while y.data.norm() < 1000:
    y  = y * 2
print('\n',x, '\n',y)
v = torch.tensor([1,1,.5])
y.backward(v)
print(x.grad)


 tensor([ 0.5926,  0.5438, -1.0933], requires_grad=True) 
 tensor([  606.7927,   556.8962, -1119.4907], grad_fn=<MulBackward0>)
tensor([1024., 1024.,  512.])


### 2. Pytorch: 定义新的autograd函数

In [91]:
import torch
from torch.autograd import Variable

class MyRelu(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return input.clamp(min = 0)
    
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input < 0] = 0
        return grad_input
    
x = torch.tensor([1,1,2,3,-1,1,-2,0], dtype = torch.float32, requires_grad = True)
relu = MyRelu.apply
y = relu(x)
y.backward(torch.ones(x.shape))
print(x.grad)

tensor([1., 1., 1., 1., 0., 1., 0., 1.])


In [106]:
import torch

class LinearFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, weight, bias = None):
        #print(type(input))
        ctx.save_for_backward(input, weight, bias)
        output = input.mm(weight.t())
        if bias is not None:
            output += bias.unsqueeze(0).expand_as(output)
        return output
    
    @staticmethod
    def backward(ctx, grad_output):
        input, weight, bias = ctx.saved_variables
        grad_input = grad_weight = grad_bias = None

        if ctx.needs_input_grad[0]:
            grad_input = grad_output.mm(weight)
        if ctx.needs_input_grad[1]:
            grad_weight = grad_output.t().mm(input)
        if bias is not None and ctx.needs_input_grad[2]:
            grad_bias = grad_output.sum(0).squeeze(0)

        return grad_input, grad_weight, grad_bias

def linear(input, weight, bias = None):
    return LinearFunction()(intput, weight, bias)

linear = LinearFunction.apply

#Check backwordb !!
input = (Variable(torch.randn(3, 5).double(), requires_grad = True),
        Variable(torch.randn(3, 5).double(), requires_grad = True),None)
test = gradcheck(linear, input, eps=1e-6, atol=1e-4)
print(test)

True


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: 'saved_variables' is deprecated; use 'saved_tensors'
  from ipykernel import kernelapp as app


In [113]:
#expand to Module 
import torch.nn as nn
class MyLinear(nn.Module):
    def __init__(self, input_features, output_features, bias = True):
        super(MyLinear, self).__init__()
        self.input_features = input_features
        self.output_features = output_features
        
        self.weight = nn.Parameter(torch.Tensor(output_features, input_features))
        if bias:
            self.bias = nn.Parameter(torch.Tensor(output_features))
        else:
            self.register_parameter('bias', None)
        self.weight.data.uniform_(-0.1, 0.1)
        if bias is not None:
            self.bias.data.uniform_(-0.1, 0.1)
    
    def forward(self, input):
        return LinearFunction.apply(input, self.weight, self.bias)

x = torch.randn(3,5)
fc1 = MyLinear(5, 4, bias= None)
output =fc1(x)
print(output)

tensor([[-0.1637,  0.1026,  0.0172,  0.1918],
        [ 0.1469,  0.0212,  0.1668, -0.0239],
        [ 0.2117, -0.0148,  0.1033,  0.0719]],
       grad_fn=<LinearFunctionBackward>)


In [124]:
from torch.autograd import Variable
a = torch.randn(10, 5)
print(a)
b = torch.randn(10, 5)
print(b)
x = Variable(a, requires_grad=True)
y = Variable(b, requires_grad=True)
z = x + y
z.backward(torch.ones(a.shape))
print(x.grad)            # x的梯度 10x1 的全1 tensor
z.grad_fn         # <SumBackward0 object at 0x7f809e33fcf8> 

tensor([[ 1.6238e-03,  1.4402e-02, -5.3506e-01, -2.5703e-01,  9.4487e-02],
        [ 2.3796e+00, -1.4976e+00,  1.4442e-01, -4.4399e-01,  1.0962e+00],
        [ 1.3845e+00, -9.0367e-01, -9.9576e-02, -1.2792e-01,  1.0103e+00],
        [-1.2547e+00,  2.4198e-01,  4.0399e-01, -2.2229e+00,  5.6809e-01],
        [ 3.9360e-01,  2.0778e+00, -3.9482e-01, -9.9242e-01,  1.7436e+00],
        [-5.0620e-01,  7.0693e-01, -4.6644e-01, -6.2862e-01,  1.6091e-02],
        [-1.8387e+00, -8.8481e-01,  1.0101e+00, -2.5769e-01,  1.9752e+00],
        [-4.1672e-01,  5.5078e-01, -5.5916e-01, -1.1018e+00, -1.5692e+00],
        [ 1.8142e+00,  7.6748e-01, -5.8636e-01,  7.6672e-02,  7.0167e-01],
        [-1.0566e-01, -3.5950e-01, -4.6776e-01, -2.2812e-01,  1.9953e+00]])
tensor([[-1.4433, -0.1183, -0.1677,  0.9031, -1.1447],
        [-0.9931, -0.2226, -2.3782, -0.6939,  0.4632],
        [ 0.5125,  0.8370, -0.4732,  0.0306,  0.8650],
        [-0.7645,  0.8148, -1.5711,  0.7891, -0.3908],
        [ 0.0653, -0.2223, -2